# Analyze phenotypic data
### Start by importing packages and loading the dataframe from the previous step

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pingouin
import seaborn as sns

df = pd.read_pickle('df.pkl')

In [4]:
# Get number of male and females in the low and high TOWRE groups
m_low_t = sum(inds_low_t) - sum(df[inds_low_t]['SEX'])
f_low_t = sum(inds_low_t) - m_low_t
m_high_t = sum(inds_high_t) - sum(df[inds_high_t]['SEX'])
f_high_t = sum(inds_high_t) - m_high_t
print(m_low_t, f_low_t, m_high_t, f_high_t)

105.0 58.0 342.0 181.0


### Check for group differences (Table 1)

In [3]:
# PRINT COHORT-WIDE STATISTICS
print('AVERAGE AGE:',np.mean(df['AGE']),'(',np.std(df['AGE']),')')
print('AVERAGE HAND:',np.mean(df['HAND']),'(',np.std(df['HAND']),')')
print('AVERAGE gFA:',np.mean(df['gFA']),'(',np.std(df['gFA']),')')
print('AVERAGE TBV:',np.mean(df['TBV']),'(',np.std(df['TBV']),') \n')

# Get the mean, sum, and std of stats broken down by groups
mean_stats_ht_lt = df.groupby('HT_LT').mean()
sum_stats_ht_lt = df.groupby('HT_LT').sum()
sd_stats_ht_lt = df.groupby('HT_LT').std()

# Get indices of high and low TOWRE participants
inds_low_t = (df['HT_LT']==1)
inds_high_t = (df['HT_LT']==0)

# DIFFERENCES IN SEX (Chi-Square)
#print('SEX: M HIGH T:',m_high_t,', F HIGH T:',f_high_t,', M LOW T',m_low_t,', F LOW T',f_low_t,'\n')

# DIFFERENCES IN AGE (Welch's t-test)
print('COMPARE AGE: HIGH T MEAN (SD):',mean_stats_ht_lt['AGE'][0],'(',sd_stats_ht_lt['AGE'][0],'), LOW T MEAN (SD):', 
      mean_stats_ht_lt['AGE'][1],'(',sd_stats_ht_lt['AGE'][1],')\n')
print(str(pingouin.ttest(df['AGE'][inds_high_t], df['AGE'][inds_low_t]))+'\n')

# DIFFERENCES IN HANDEDNESS (Welch's t-test)
print('COMPARE HAND: HIGH T MEAN (SD):',mean_stats_ht_lt['HAND'][0],'(',sd_stats_ht_lt['HAND'][0],'), LOW T MEAN (SD):', 
      mean_stats_ht_lt['HAND'][1],'(',sd_stats_ht_lt['HAND'][1],')\n')
print(str(pingouin.ttest(df['HAND'][inds_high_t], df['HAND'][inds_low_t]))+'\n')

# DIFFERENCES IN gFA (Welch's t-test)
print('COMPARE gFA: HIGH T MEAN (SD):',mean_stats_ht_lt['gFA'][0],'(',sd_stats_ht_lt['gFA'][0],'), LOW T MEAN (SD):', 
      mean_stats_ht_lt['gFA'][1],'(',sd_stats_ht_lt['gFA'][1],')\n')
print(str(pingouin.ttest(df['gFA'][inds_high_t], df['gFA'][inds_low_t]))+'\n')

# DIFFERENCES IN TBV (Welch's t-test)
print('COMPARE TBV: HIGH T MEAN (SD):',mean_stats_ht_lt['TBV'][0],'(',sd_stats_ht_lt['TBV'][0],'), LOW T MEAN (SD):', 
      mean_stats_ht_lt['TBV'][1],'(',sd_stats_ht_lt['TBV'][1],')\n')
print(str(pingouin.ttest(df['TBV'][inds_high_t], df['TBV'][inds_low_t])))

AVERAGE AGE: 10.69410603498543 ( 3.0921610553177366 )
AVERAGE HAND: 61.77911078717179 ( 50.69374008308745 )
AVERAGE gFA: 0.3761593997478485 ( 0.051086898893117905 )
AVERAGE TBV: 2183975.252186589 ( 325109.32658847165 ) 

COMPARE AGE: HIGH T MEAN (SD): 10.857504973231357 ( 3.2002155952461298 ), LOW T MEAN (SD): 10.169826006134969 ( 2.669435882547439 )

               T         dof alternative     p-val         CI95%   cohen-d  \
T-test  2.733298  319.722849   two-sided  0.006619  [0.19, 1.18]  0.223071   

       BF10     power  
T-test  3.7  0.699608  

COMPARE HAND: HIGH T MEAN (SD): 61.38483747609943 ( 49.45588498740397 ), LOW T MEAN (SD): 63.04417177914111 ( 54.76249625491039 )

               T         dof alternative    p-val          CI95%   cohen-d  \
T-test -0.345432  249.814807   two-sided  0.73006  [-11.12, 7.8]  0.032688   

         BF10     power  
T-test  0.106  0.065302  

COMPARE gFA: HIGH T MEAN (SD): 0.37733173 ( 0.05275477188563018 ), LOW T MEAN (SD): 0.37239838 ( 0.

### Plot TOWRE Distribution (Figure 1)

In [ ]:
font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 22}
sns.set(font_scale=1.3)
sns.set_style("white")
plt.rc('font', **font)
t_plot = sns.displot(df, x="TOWRE", hue="TR_RD",bins=20)
t_plot._legend.remove()
plt.vlines(85,0,90,'k',linewidth=5,linestyle=':')
plt.ylim([0,80])
plt.xlabel('TOWRE Total Score, Standardized',fontsize=20,fontweight='bold')
plt.ylabel('Number of Participants',fontsize=20,fontweight='bold')
plt.savefig('TOWRE.pdf') # use Adobe Illustrator to convert to EPS in order to keep transparent background
plt.show()